In [1]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [2]:
import pandas as pd

from src import value_filler as vs
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [92]:
def get_performance(model, X_train, X_test, y_train, y_test):
    y_pred = model.predict(X_test)
    roc_auc_train = roc_auc_score(y_train, model.predict(X_train))
    # Validation ROC AUC Score
    acc = accuracy_score(y_test, y_pred)
    roc_auc_val = roc_auc_score(y_test, y_pred)
    print( f"Model Accuracy: {acc}",
            f"Train roc_auc: {roc_auc_train}", 
            f"Test roc_auc: {roc_auc_val}",
            sep='\n', end='\n\n')
    print(confusion_matrix(y_test, y_pred),
    classification_report(y_test, y_pred),
    sep='\n'
)

In [93]:
n=2
df=pd.read_csv(f'data/waves/wave{n}_raw.csv')
target=f'r{n}hosp1y'
drop=f'r{n}hspnit1y'

In [94]:
# Drope
df = df.drop([drop], axis=1)

df = df.dropna(subset=[target])

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [95]:
# filling
df_f = fs.fast_fill(df)

In [97]:
# split
y=df_f[target]
X=df_f.drop(target,axis=1)

X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [98]:
# resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.35)
X_test, y_test = dp.apply_resample(X_test, y_test,v=1)

In [99]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [100]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [101]:
# power transform
X_train, X_test = dp.apply_power_transform(X_train, X_test)

In [102]:
import xgboost as xgb
from sklearn.model_selection import KFold

def train_xgboost_with_kfold(features, labels, model):
    # Definir el número de folds (k)
    k = 7
    
    kf = KFold(n_splits=k)
    
    # Iterar sobre los folds
    for train_index, test_index in kf.split(features):
        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        # Entrenar el modelo XGBoost
        model.fit(X_train, y_train)
    
    return model

model_untrained = xgb.XGBClassifier()
model= train_xgboost_with_kfold(X_train, y_train,model_untrained)

get_performance(model, X_train, X_test, y_train, y_test)

Model Accuracy: 0.859271523178808
Train roc_auc: 0.9964328180737217
Test roc_auc: 0.859271523178808

[[257  45]
 [ 40 262]]
              precision    recall  f1-score   support

         0.0       0.87      0.85      0.86       302
         1.0       0.85      0.87      0.86       302

    accuracy                           0.86       604
   macro avg       0.86      0.86      0.86       604
weighted avg       0.86      0.86      0.86       604



In [103]:
model_2 = xgb.XGBClassifier()
model_2 .fit(X_train, y_train)
get_performance(model_2, X_train, X_test, y_train, y_test)

Model Accuracy: 0.8609271523178808
Train roc_auc: 1.0
Test roc_auc: 0.8609271523178808

[[264  38]
 [ 46 256]]
              precision    recall  f1-score   support

         0.0       0.85      0.87      0.86       302
         1.0       0.87      0.85      0.86       302

    accuracy                           0.86       604
   macro avg       0.86      0.86      0.86       604
weighted avg       0.86      0.86      0.86       604

